In [1]:
import pandas as pd
import numpy as np
import re
import jieba
import jieba.posseg as psg

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [19]:
data = pd.read_excel("../data/data_a.xlsx")
dic_file = "../data/test_dict.txt"
stop_file = "../data/stopwords.txt"

In [20]:
def chinese_word_cut(mytext):
    jieba.load_userdict(dic_file)
    jieba.initialize()
    try:
        stopword_list = open(stop_file, 'r', encoding='utf-8')
    except:
        stopword_list = []
    
    stop_list = []
    flag_list = ['n', 'nz', 'vn']
    word_list = []

    for line in stopword_list:
        line = re.sub(u'\n|\\r', '', line)
        stop_list.append(line)

    #jieba分词
    seg_list = psg.cut(mytext)
    for seg_word in seg_list:
        word = re.sub(u'[^\u4e00-\u9fa5]','',seg_word.word)
        #word = seg_word.word  #如果想要分析英语文本，注释这行代码，启动下行代码
        find = 0
        for stop_word in stop_list:
            if stop_word == word or len(word) < 2:     #this word is stopword
                    find = 1
                    break
        if find == 0 and seg_word.flag in flag_list:
            word_list.append(word) 
         
    return (" ").join(word_list)

In [18]:
chinese_word_cut("看#韩素希三问李惠利#有感：#同为女生不该为了臭男生吵架#。”#韩素希指责柳俊烈沉默# 新浪娱乐的微博视频")

'女生 男生 娱乐'

In [21]:
data['content_cutted'] = data.content.apply(chinese_word_cut)

In [22]:
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

In [23]:
n_features = 1000 #提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(data.content_cutted)

In [24]:
n_topics = 7
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50,
#                                 doc_topic_prior=0.1,
#                                 topic_word_prior=0.01,
                               random_state=0)
lda.fit(tf)

LatentDirichletAllocation(learning_offset=50, max_iter=50, n_components=7,
                          random_state=0)

In [25]:
n_top_words = 25
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
比赛 球队 火箭 篮板 球员 时间 体育 助攻 奇才 训练 球迷 湖人 体育讯 战胜 问题 主场 状态 俱乐部 领先 机会 内线 时候 火箭队 男篮 对阵
Topic #1:
孩子 研究 银行 时间 新闻 人类 科学家 工作 网友 工作人员 生活 男子 女孩 老师 任务 大家 妈妈 女儿 女生 小孩 开学 小时 人们 年轻人 信用卡
Topic #2:
专家 网友 老师 小米 压力 汽车 黄金 走势 分析 股票 突破 问题 大盘 成本 价格 市场 调整 整理 建议 股价 股市 机会 趋势 大家 坐堂
Topic #3:
公司 苹果 用户 产品 快讯 市场 平台 记者 网络 业务 发展 科技 手机 信息 内容 品牌 网站 互联网 消费者 技术 服务 消费 行业 销售 地震
Topic #4:
电影 主持人 大家 票房 时候 影片 娱乐 观众 演员 故事 作品 现场 朋友 角色 合作 人生 感觉 电影节 市场 考研 主演 时间 公司 生活 内地
Topic #5:
经济 政府 计划 国家 公司 总统 投资 记者 问题 时间 国际 企业 全球 发展 员工 事件 人员 世界 市场 媒体 新闻 主席 地带 政策 项目
Topic #6:
学生 学校 大学 教育 建议 工作 问题 专业 国家 教师 基金 全国 移民 委员 社会 情况 留学生 语言 政策 成绩 影响 课程 家庭 女性 孩子


In [26]:
topics=lda.transform(tf)

In [27]:
topic = []
for t in topics:
    topic.append("Topic #"+str(list(t).index(np.max(t))))
data['概率最大的主题序号']=topic
data['每个主题对应概率']=list(topics)
data.to_excel("data_topic.xlsx",index=False)